<a href="https://colab.research.google.com/github/myrandaGoesToSpace/semantic-search-datasets/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install bs4

     |████████████████████████████████| 78 kB 2.0 MB/s 
     |████████████████████████████████| 3.5 MB 5.4 MB/s 
     |████████████████████████████████| 6.8 MB 17.2 MB/s 
     |████████████████████████████████| 1.2 MB 30.0 MB/s 
     |████████████████████████████████| 67 kB 1.1 MB/s 
     |████████████████████████████████| 895 kB 18.8 MB/s 
     |████████████████████████████████| 596 kB 22.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=6395bcb34ad69007c358b0a26a13f0bb218c169762ac2cb6ea2703eabd6c96ec
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer, util
from bs4 import BeautifulSoup

In [ ]:
'''
# Example - semantic search

query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)


#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)
  
'''

In [ ]:
import pandas as pd
import requests

def find_abstracts(soup):
  #df = pd.DataFrame(columns = ["identifier", "abstract"])
  id_list = []
  abs_list = []
  title_list = []

  for record in soup.find_all("csw:record"):
    id = record.find("dc:identifier")
    abs = record.find("dct:abstract")
    title = record.find("dc:title")

    # append id and abs to df
    #df = df.append([id.text, abs.text])
    id_list.append(id.text)
    title_list.append(title.text)

    if abs != None:
      abs_list.append(abs.text)
    else:
      abs_list.append("NA")

  return id_list, title_list, abs_list

# Get the abstracts from Geoportal
URL = "https://www.ncei.noaa.gov/metadata/geoportal/opensearch?f=csw&from=0&size=5000&sort=title.sort"

page = requests.get(URL)
soup = BeautifulSoup(page.text, "lxml")

id_list, title_list, abs_list = find_abstracts(soup)
df = pd.DataFrame(list(zip(id_list,title_list, abs_list)), columns = ["identifier", "title", "abstract"])
df

,identifier,title,abstract
0,gov.noaa.nodc:0000662,(NCEI Accession 0000662),NA
1,gov.noaa.nodc:0208737,(NCEI Accession 0208737),NA
2,gov.noaa.nodc:0215155,(NCEI Accession 0215155),NA
3,gov.noaa.ngdc.mgg.geophysics:G01414,1-deg x 1-deg Terrestrial Mean Free-Air Anomalies,The 1x1 degree Terrestrial Mean Free-Air Gravi...
4,gov.noaa.nodc:0002166,11 Real-time XBT replacements assembled by Can...,NA
...,...,...,...
4995,gov.noaa.ncei:NA127_EM302,EM302 Water Column Sonar Data Collected During...,Santa Barbara Basin and West Coast EEZ Mapping.
4996,gov.noaa.ncei:NA128_EM302,EM302 Water Column Sonar Data Collected During...,Cascadia Margin Seep Exploration
4997,gov.noaa.ngdc.mgg.wcd:SKQ201401S_EM302,EM302 Water Column Sonar Data Collected During...,Recovery of Seamount Precious Coral Beds (SKQ2...
4998,gov.noaa.ngdc.mgg.wcd:SKQ201402S_EM302,EM302 Water Column Sonar Data Collected During...,SKQ201402S Cruise (EM302). The basic premise o...


In [ ]:

# Make the abstracts the docs
docs_df = df[df["abstract"] != "NA"]
docs = list(docs_df["abstract"])
titles = list(docs_df["title"])

# Query
query = input("Enter your query: ")

# predict on a search query for data

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores, titles))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)


#Output passages & scores
for doc, score, title in doc_score_pairs[:10]:
    print("Score: %f" %score)
    print("Title: %s" %title)
    print("Abstract: %s" %doc)
    print('\n')

Enter your query: coral health in the Gulf of Mexico
Score: 0.591545
Title: Digital Video taken during the 3-person submersible Clelia dive 621 of the NOAA Office of Ocean Exploration's Islands in the Stream 2001 cruise, 2001-09-05 (NCEI Accession 0039428)
Abstract: Islands in the Stream is a three-month scientific expedition to marine protected areas and other habitats being considered for protection from offshore of Belize in Central America to North Carolina’s continental shelf. "Islands” are the coral reefs and hard-bottom (also called “hard substrate”) biological communities found along the course of a massive “stream” of ocean currents connecting the Caribbean Sea, the Gulf of Mexico, and the western Atlantic off North America. This expedition emphasizes habitat characterization and an understanding of these islands' "connection" as parts of an interrelated ecosystem. Though separated by large expanses of ocean water, the fishes, corals, and invertebrates common to these islands 